# Topic Model Parts of Speech

This is a notebook for trying to use topic models for classifying sets of text that are more syntactically similar than topically similar. This notebook attempts to distinguish between discussion and conclusion section of scientific papers.

Below we are loading the dataset for use.

In [1]:
from __future__ import print_function
from time import time
import os

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cross_validation import train_test_split

import numpy as np

import pickle

validDocsDict = dict()
fileList = os.listdir("BioMedPOS")
for f in fileList:
    validDocsDict.update(pickle.load(open("BioMedPOS/" + f, "rb")))

Here we are setting some vaiables to be used below and defining a function for printing the top words in a topic for the topic modeling.

In [2]:
n_samples = len(validDocsDict.keys())
n_features = 200
n_topics = 2
n_top_words = 10


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Pre-process data

Here we are preprocessing data for use later. This code only grabs the discussion and conclusion sections of the data. We are also creating appropriate labels for the data and spliting the documents up to train and test sets.

In [3]:
print("Loading dataset...")
t0 = time()
documents = []

labels = []
concLengthTotal = 0
discLengthTotal = 0
concCount = 0
discCount = 0

for k in validDocsDict.keys():
    if k.startswith("conclusion"):
        labels.append("conclusion")
        documents.append(validDocsDict[k])
        concCount += 1
        concLengthTotal += len(validDocsDict[k].split(' '))
    elif k.startswith("discussion"):
        labels.append("discussion")
        documents.append(validDocsDict[k])
        discCount += 1
        discLengthTotal += len(validDocsDict[k].split(' '))

print(len(documents))
print(concLengthTotal * 1.0/ concCount)
print(discLengthTotal * 1.0/ discCount)

train, test, labelsTrain, labelsTest = train_test_split(documents, labels, test_size = 0.1)

Loading dataset...
49698
728.293895127
1408.85472252


Here we are splitting the data up some more to train different models. Discussion and conclusion sections are being put into their own training sets. A TFIDF vectorizer is trained with the whole dataset of conclusion AND discussion sections. The multiple different training sets are then transformed using this vectorizer to get vector encodings of the text normalized to sum to 1 which accounts for differing lengths of conclusion and discussion sections.

In [4]:
trainSetOne = []
trainSetTwo = []

for x in range(len(train)):
    if labelsTrain[x] == "conclusion":
        trainSetOne.append(train[x])
    else:
        trainSetTwo.append(train[x])

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
#tf_vectorizer = TfidfVectorizer(max_df=0.95, norm = 'l1', min_df=2, max_features=n_features)
tf_vectorizer = TfidfVectorizer(max_df=0.95, norm = 'l1', min_df=2, max_features=n_features, ngram_range = (1,4))
t0 = time()
tf = tf_vectorizer.fit_transform(train)

tfSetOne = tf_vectorizer.transform(trainSetOne)
tfSetTwo = tf_vectorizer.transform(trainSetTwo)
tfTest = tf_vectorizer.transform(test)
test = tfTest
train = tf
trainSetOne = tfSetOne
trainSetTwo = tfSetTwo

print("done in %0.3fs." % (time() - t0))

Extracting tf features for LDA...
done in 355.574s.


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


# LDA With Two Topics

Define an LDA topic model on the whole data set with two topics. This is trying to see if the topic model can define the difference between the two groups automatically and prints the top words per topic.

In [5]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=100,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Fitting LDA models with tf features, n_samples=147613 and n_features=200...
done in 329.437s.

Topics in LDA model:
Topic #0:
cd nnp nnp nnp cd nn nn nn nn nn cd cd cd nnp nnp nnp jj nn nn nn cd nn
Topic #1:
in dt nn vbp vbd dt nn in nn in dt prp vbn in vbg nn in nn in dt jj


Transform the unknown data through the topic model and calculate which topic it is more associated with according to the ratios. Calculate how many of each type (conclusion and discussion) go into each topic (1 or 2).

In [6]:
results = lda.transform(test)
totalConTop1 = 0
totalConTop2 = 0
totalDisTop1 = 0
totalDisTop2 = 0
for x in range(len(results)):
    val1 = results[x][0]
    val2 = results[x][1]
    total = val1 + val2
    print(str(labelsTest[x]) + " " + str(val1/total) + " " + str(val2/total))
    if val1 > val2:
        if labelsTest[x] == "conclusion":
            totalConTop1 += 1
        else:
            totalDisTop1 += 1
    else:
        if labelsTest[x] == "conclusion":
            totalConTop2 += 1
        else:
            totalDisTop2 += 1

conclusion 0.629083807578 0.370916192422
discussion 0.346218836732 0.653781163268
discussion 0.371972766247 0.628027233753
discussion 0.401238421662 0.598761578338
discussion 0.410752453511 0.589247546489
conclusion 0.656804245273 0.343195754727
discussion 0.435077812131 0.564922187869
conclusion 0.640681564985 0.359318435015
discussion 0.379160029856 0.620839970144
discussion 0.46773705721 0.53226294279
discussion 0.527141476323 0.472858523677
discussion 0.402019042265 0.597980957735
discussion 0.363213261829 0.636786738171
discussion 0.374525068383 0.625474931617
conclusion 0.638614789402 0.361385210598
discussion 0.36679914852 0.63320085148
discussion 0.405812654629 0.594187345371
conclusion 0.642753574011 0.357246425989
conclusion 0.71971088375 0.28028911625
conclusion 0.532231195423 0.467768804577
conclusion 0.321151047973 0.678848952027
discussion 0.369152713117 0.630847286883
conclusion 0.663743363632 0.336256636368
discussion 0.400904216281 0.599095783719
conclusion 0.359587231

Print out the results from the topic transforms.

In [7]:
print("Total Conclusion Topic One: " + str(totalConTop1))
print("Total Conclusion Topic Two: " + str(totalConTop2))
print("Total Discussion Topic One: " + str(totalDisTop1))
print("Total Discussion Topic Two: " + str(totalDisTop2))

Total Conclusion Topic One: 1570
Total Conclusion Topic Two: 893
Total Discussion Topic One: 324
Total Discussion Topic Two: 2183


Get the parameters for the LDA.

In [8]:
lda.get_params()

{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 50.0,
 'max_doc_update_iter': 100,
 'max_iter': 100,
 'mean_change_tol': 0.001,
 'n_jobs': 1,
 'n_topics': 2,
 'perp_tol': 0.1,
 'random_state': 0,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}

# Basic Classifiers

Train three basic classifiers to solve the problem. Try Gaussian, Bernoulli and K Nearest Neighbors classifiers and calculate how accurate they are.

In [9]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(train.toarray(), labelsTrain)

classResults = classifier.predict(test.toarray())

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.949698189135


In [10]:
from sklearn.naive_bayes import BernoulliNB

classifier = BernoulliNB()

classifier.fit(train.toarray(), labelsTrain)

classResults = classifier.predict(test.toarray())

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.829175050302


In [11]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()

classifier.fit(train, labelsTrain)

classResults = classifier.predict(test)
numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.828370221328


# Two Topic Models

Define two topic models with 20 topics each, one on discussion sections and one on conclusion sections. Then transform both the train and test sets using both topic models to get 40 features for each sample based on the probability distribution for each topic in each LDA.

In [12]:
ldaSet1 = LatentDirichletAllocation(n_topics=20, max_iter=100,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
ldaSet2 = LatentDirichletAllocation(n_topics=20, max_iter=100,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

In [13]:
ldaSet1.fit(trainSetOne)
print_top_words(ldaSet1, tf_feature_names, n_top_words)

Topic #0:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #1:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #2:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #3:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #4:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #5:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #6:
vbp in dt nn vbg dt nn in nn in nn prp nn in dt nnp nn nn nn jj jj
Topic #7:
cd cd cd cd cd nn cd cd cd cd cd nn cd cd cd nn cd nn cd nn nn cd cd cd nn nn nn cd nn cd vbd rb rb dt
Topic #8:
cd nnp nnp nnp nn cd nn nn nn cd nn cd cd nnp nnp nnp nn cd nn jj nn nn
Topic #9:
cd cd cd cd cd nn cd cd cd cd cd nn cd c

In [14]:
ldaSet2.fit(trainSetTwo)
print_top_words(ldaSet2, tf_feature_names, n_top_words)

Topic #0:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp nnp nn cd nn cd nnp nn nn nnp nnp nnp
Topic #1:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp nnp nn cd nn cd nnp nn nn nnp nnp nnp
Topic #2:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp nnp nn cd nn cd nnp nn nn nnp nnp nnp
Topic #3:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp nnp nn cd nn cd nnp nn nn nnp nnp nnp
Topic #4:
cd vbd in dt nn cd nn dt nn in vbp nn in dt nn in nn prp vbn in
Topic #5:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp nnp nn cd nn cd nnp nn nn nnp nnp nnp
Topic #6:
cd nnp cd nn nn cd vbd cd cd nnp nnp in dt nn nn nn nn nn in nn
Topic #7:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp nnp nn cd nn cd nnp nn nn nnp nnp nnp
Topic #8:
cd cd cd cd nn nnp nnp nnp nnp nnp nnp nn nn nnp nnp nnp nn cd nn cd cd in nnp n

In [15]:
results1 = ldaSet1.transform(train)
results2 = ldaSet2.transform(train)

resultsTest1 = ldaSet1.transform(test)
resultsTest2 = ldaSet2.transform(test)

In [16]:
results = np.hstack((results1, results2))
resultsTest = np.hstack((resultsTest1, resultsTest2))

Define two classifiers using the transformed train and test sets from the topic models. Print out the accuracy of each one.

In [17]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.745472837022


In [18]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.871227364185


Normalize the results of each sample of 40 features so they sum to 1. Then train two more classifiers using the data and print out the accuracy of each.

In [19]:
for x in range(len(results)):
    total = 0
    for y in range(len(results[x])):
        total += results[x][y]
    for y in range(len(results[x])):
        results[x][y] = results[x][y]/total
        
for x in range(len(resultsTest)):
    total = 0
    for y in range(len(resultsTest[x])):
        total += resultsTest[x][y]
    for y in range(len(resultsTest[x])):
        resultsTest[x][y] = resultsTest[x][y]/total

In [20]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.495573440644


In [21]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()

classifier.fit(results, labelsTrain)

classResults = classifier.predict(resultsTest)

numRight = 0

for item in range(len(classResults)):
    if classResults[item] == labelsTest[item]:
        numRight += 1

print(str(numRight * 1.0 / len(classResults) * 1.0))

0.871227364185
